Universidade Federal do Rio Grande do Sul (UFRGS)   
Programa de Pós-Graduação em Engenharia Civil (PPGEC)   

# Project PETROBRAS (2018/00147-5): 
## Attenuation of dynamic loading along mooring lines embedded in clay

---
_Prof. Marcelo M. Rocha, Dr.techn._ [(ORCID)](https://orcid.org/0000-0001-5640-1020)  
Porto Alegre, RS, Brazil 
___

[1.   Introduction](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/01_Introduction.ipynb?flush_cache=true)  
[2.   Reduced model scaling](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/02_Reduced_model.ipynb?flush_cache=true)  
[3.   Typical soil](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/03_Typical_soil.ipynb?flush_cache=true)  
[4.   The R4 studless 120mm chain](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/04_R4_studless_chain.ipynb?flush_cache=true)  
[5.   Dynamic load definition](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/05_Dynamic_load.ipynb?flush_cache=true)  
[6.   Design of chain anchoring system](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/06_Chain_anchor.ipynb?flush_cache=true)  
[7.   Design of uniaxial load cell with inclinometer](https://nbviewer.jupyter.org/github/mmaiarocha/Attenuation/blob/master/07_Load_cell.ipynb?flush_cache=true)  


In [1]:
# Importing Python modules required for this notebook
# (this cell must be executed with "shift+enter" before any other Python cell)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing "pandas dataframe" with dimension exponents for scales calculation
DimData = pd.read_excel('resources/DimData.xlsx')
print(DimData)

                       descriptor   latex  L  M  T
a                    Acceleration       a  1  0 -2
α            Angular acceleration  \alpha  0  0 -2
ω               Angular frequency  \omega  0  0 -1
A                            Area       A  2  0  0
EI         Beam bending stiffness      EI  0  1 -2
GAs          Beam shear stiffness    GA_s  2  1 -2
ρ                         Density    \rho -3  1  0
μ               Dynamic viscosity     \mu -1  1 -1
F                           Force       F  1  1 -2
f                       frequency       f  0  0 -1
υ             Kinematic viscosity     \nu  2  0 -1
L                          Length       L  1  0  0
m                            Mass       m  0  1  0
im   Mass inertia per unit length     i_m  1  1  0
μA             Mass per unit area   \mu_A -2  1  0
μL           Mass per unit length   \mu_L -1  1  0
M                          Moment       M  2  1 -2
I               Moment of inertia       I  4  0  0
W               Resistent modul

## 2. Reduced model scaling   
[(Link for PEC00144 class on Dimensional Analysis)](https://nbviewer.jupyter.org/github/mmaiarocha/PEC00144/blob/master/1_Dimensional_Analysis.ipynb)

Experiments must be designed with a reduced length scale specified as $\lambda_L = 1:10$. 
By considering that the soil in the experimental field satisfactorily resembles the 
specified _typical soil_ (see [section 3](https://nbviewer.jupyter.org/urls/dl.dropbox.com/s/z35uz1iz5be4mq2/03_Typical_soil.ipynb?flush_cache=true)), the further scale-restricted 
quantities are the soil density, $\rho_{\rm soil} = 1600{\rm kg/m^3}$ with scale $\lambda_\rho = 1:1$, 
and the gravity acceleration, $g = 9.81{\rm m/s^2}$ also with scale $\lambda_g = 1:1$. 
All other quantities have derived scales that must be calculated and used for the 
interpretation of experimental results. 

The choice of these three control quantities, $L$, $\rho$, and $g$, is based on the very
basic assumption that _the most important parameter governing the soil reaction for large
displacements is the undrained shear resistance_, $s_{\rm u}$. This parameter is assumed
to have the general form:

$$ s_{\rm u} = k\rho_{\rm soil} g z $$

where $k$ is a non-dimensional factor and $z$ the depth measured from soil surface.
Under this assumption, the scale of $s_{\rm u}$ will be correct at any depth $z$.

The three control quantities allow the definition of an new dimensional base to be used 
for calculating the derived scales of further relevant quantities. 
Dimension exponents are read from ``pandas`` dataframe ``DimData``:


In [5]:
ABC  = ['L', 'rho',   'α']     # control quantities are length, density and acceleration
LMT  = ['L',   'M',   'T']     # dimension exponents (last three columns of DimData dataframe)

base =  DimData.loc[ABC, LMT];   print(base)

     L  M  T
L    1  0  0
rho -3  1  0
α    0  0 -2


Scales calculation requires the inversion of this new base, what is carried out with 
``numpy`` method ``linalg.inv``:

In [3]:
i_base = np.linalg.inv(base.values);   print(i_base)

[[ 1.  -0.  -0. ]
 [ 3.   1.   0. ]
 [ 0.5 -0.  -0.5]]


Now we specify a list of all quantities for which derived scales are to be calculated. 
We choose, for instance:
 
* Force, $F$, (for chain tension),
* Frequency, $f$, (for dynamic loading spectral density),
* Mass per unit length, $\mu_L$, (for specifying the grade of model chain),
* Stress, $\sigma$, (for soil resistance)  
 
A list with the identifiers of these quantities is used to read their dimension 
exponents from ``DimData`` dataframe, given the problem dimension matrix also as
a dataframe:

In [4]:
param  = ['F', 'f', 'mL', 'sigma']      # parameters which scales must be calculated
npar   = len(param)                     # number of parameters in the previous list

DimMat =  DimData.loc[param, LMT];   print(DimMat)

       L  M  T
F      1  1 -2
f      0  0 -1
mL    -1  1  0
sigma -1  1 -2


The code snipet below calculates a new dimension matrix with the dimension exponents 
corresponding to the new base. This new matrix is directly formatted as a ``pandas`` dataframe:

In [5]:
NewMat = pd.DataFrame(data    = np.dot(DimMat,i_base),
                      index   = DimMat.index,
                      columns = ABC)
print(NewMat)

         L  rho    a
F      3.0  1.0  1.0
f     -0.5  0.0  0.5
mL     2.0  1.0  0.0
sigma  1.0  1.0  1.0


To check the results above, let us take a look in the force dimensions:
 
\begin{align*}
 [F] &= [L]^3 \, [\rho]^1 \, [a]^1 \\
     &= {\rm (m)^3 \, (kg/m^3)^1 \, (m/s^2)^1} \\
     &= {\rm kg \, m \, / \, s^2} \\
     &= {\rm N}
\end{align*} 

where $[\cdot]$ means ''unit of''. One may conclude that the computational procedure is _ok_, 
despite its conciseness.

The next step is the especification of experimental scales for the control quantities, as
previously discussed:

In [6]:
lbd_length  =  1/10      # length scale for the reduced model
lbd_density =  1/1       # same soil, with same density
lbd_accel   =  1/1       # gravity will not be changed!

scales      =  np.tile([lbd_length, lbd_density, lbd_accel],(npar,1))

A last code line calculates the derived scales and includes them as an additional column in
dataframe ``NewMat``:

In [7]:
NewMat['scale'] = np.prod(scales**NewMat[ABC], axis=1);   print(NewMat)

         L  rho    a     scale
F      3.0  1.0  1.0  0.001000
f     -0.5  0.0  0.5  3.162278
mL     2.0  1.0  0.0  0.010000
sigma  1.0  1.0  1.0  0.100000


where it can be seen, for instance, that the forces in the reduced model will be 
one thousandth of the real scale forces. On the other hand, model time passes $\approx$3.16 times
faster, what makes frequencies the same amount higher. 

The stress scale calculated above applies to the undrained soil shear resistance, $s_{\rm u}$, 
for this resistance depends on the product of the base quantities. However, elastic stresses
and stiffness properties (Young's and shear modulae) _are not expected to follow this
scale_. It is important to keep in mind that these quantities are not likely to be relevant
for the experimental results, for large plastic displacements are to dominate the process.

These scales will be used in the following sections to design all experimental features.